In [1]:
import numpy as np
import pandas as pd

In [2]:
congFile = '/home/hassaan/Data/TUM/2nd Year/4th Semester/IDP/Project/data/intermediate/consolidated_end116.csv'
firmFile = '/home/hassaan/Data/TUM/2nd Year/4th Semester/IDP/Project/data/intermediate/comp_firms.csv'


In [3]:
congDF = pd.read_csv(congFile)
congDF = congDF.drop(['Unnamed: 0'], axis=1)
congDF.head()

,cid,personName,year,orgid,orgname,industry,asset_type,min,max,hidemax,type,lobbies,contributes,folderOfOrigin
0,N00000939,Nadler-Jerrold,2014,D000000103,JPMorgan Chase & Co,Commercial Banks,NaN,15001,50000,0,Y,Y,NaN,cong111
1,N00000939,Nadler-Jerrold,2014,D000021792,HSBC Bank,Commercial Banks,Cash/Money Market/Savings/Checking,1001,15000,0,Y,NaN,NaN,cong111
2,N00000939,Nadler-Jerrold,2014,NaN,Congressional Federal Credit Union Account,Unknown,Cash/Money Market/Savings/Checking,1001,15000,0,Y,NaN,NaN,cong111
3,N00000939,Nadler-Jerrold,2011,D000021792,HSBC Bank,Commercial Banks,Cash/Money Market/Savings/Checking,15001,50000,0,Y,NaN,NaN,cong111
4,N00000939,Nadler-Jerrold,2011,D000000103,JPMorgan Chase & Co,Commercial Banks,NaN,1001,15000,0,Y,Y,NaN,cong111


In [4]:
firmDF = pd.read_csv(firmFile)
firmDF.head()

,gvkey,conm,conml,state
0,1000,A & E PLASTIK PAK INC,A & E Plastik Pak Inc,NaN
1,1001,A & M FOOD SERVICES INC,A & M Food Services Inc,OK
2,1002,AAI CORP,AAI Corp,MD
3,1003,A.A. IMPORTING CO INC,A.A. Importing Co Inc,MO
4,1004,AAR CORP,AAR Corp,IL


In [5]:
print(len(firmDF['gvkey'].unique()))
print(len(firmDF['conm'].unique()))
print(firmDF['conm'].describe())
print(len(firmDF))
firmDF.describe()

32813
32810
count           32813
unique          32810
top       LUMENIS LTD
freq                2
Name: conm, dtype: object
32813


,gvkey
count,32813.000000
mean,58914.747783
std,66792.343541
min,1000.000000
25%,9787.000000
50%,23959.000000
75%,115960.000000
max,319507.000000


In [6]:
firmDF[firmDF['conm']=='LUMENIS LTD']

,gvkey,conm,conml,state
14777,19845,LUMENIS LTD,Lumenis Ltd,NaN
21697,62471,LUMENIS LTD,Lumenis Ltd,NaN


In [7]:
print(congDF['cid'].unique())
print(len(congDF['cid'].unique()))
print(len(congDF['folderOfOrigin'].unique()))

['N00000939' 'N00004227' 'N00005178' ... 'N00030418' 'N99999922'
 'N00000286']
1012
12


In [8]:
print(congDF['personName'].describe())

count         245582
unique          1012
top       Kerry-John
freq            4873
Name: personName, dtype: object


In [9]:
congDF.describe()

,year,min,max,hidemax
count,245582.000000,2.455820e+05,2.455820e+05,245582.000000
mean,2009.709201,1.423359e+05,3.642206e+05,0.010819
std,3.367244,2.100153e+06,2.941882e+06,0.103451
min,2004.000000,-3.547540e+05,-3.547540e+05,0.000000
25%,2007.000000,1.001000e+03,1.500000e+04,0.000000
50%,2010.000000,1.500100e+04,3.986150e+04,0.000000
75%,2013.000000,5.000100e+04,1.000000e+05,0.000000
max,2015.000000,3.120000e+08,3.120000e+08,1.000000


### Testing string matches

#### OBSERVATIONS:
- Do the process.extract() & then run a fuzzy match on ratio and partial ratio(this checks for substring)
  for added netting

In [10]:
congOrgs = congDF['orgname'].unique()
firmOrgs = firmDF['conm'].unique()
firmOrgsLC = firmDF['conml'].unique()

In [11]:
print('unique cong orgs:', len(congOrgs))
print('unique firm orgs:', len(firmOrgs))
print('unique firm orgs lower case:', len(firmOrgsLC))

unique cong orgs: 70512
unique firm orgs: 32810
unique firm orgs lower case: 32682


In [12]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy import utils
from fuzzywuzzy.string_processing import StringProcessor

In [13]:
searchSTR = congOrgs[5000]
print(searchSTR.lower())
print('\n')
print(process.extract(searchSTR.lower(), firmOrgs, limit=5))
print('\n')
print(process.extract(searchSTR.lower(), firmOrgsLC, limit=5))

university of new mexico rev bond


[('ALLTEL NEW YORK INC', 86), ('INVESCO BOND FUND', 86), ('DIVERSICARE CP OF AMER', 86), ('GALLERY OF HISTORY INC', 86), ('NEW AMERICAN SHOE CO', 86)]


[('Alltel New York Inc', 86), ('Invesco Bond Fund', 86), ('VanEck Vectors Emerging Markets High Yield Bond ETF', 86), ('Gallery of History Inc', 86), ('New American Shoe Co', 86)]


In [14]:
orgMatch, per = process.extractOne(searchSTR, firmOrgs)
orgMatchLC, per = process.extractOne(searchSTR, firmOrgsLC)
print(searchSTR, orgMatch ,fuzz.ratio(searchSTR, orgMatch))
print(searchSTR, orgMatchLC ,fuzz.ratio(searchSTR, orgMatchLC))

University of New Mexico Rev Bond ALLTEL NEW YORK INC 19
University of New Mexico Rev Bond Alltel New York Inc 35


In [15]:
firmDF[firmDF['conml']=='CACI International Inc.']# firmDF[firmDF['conm']=='ABVIVA INC']

,gvkey,conm,conml,state
1491,2596,CACI INTL INC -CL A,CACI International Inc.,VA


In [16]:
fuzz.partial_ratio('CACI International', 'CACI International Inc.') # fuzz.partial_ratio('Citibank Cash Reserves Account 27', 'CityBank')

100

In [17]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

def levRatio(s1, s2):
    l = levenshteinDistance(s1, s2)
    m = max(len(s1), len(s2))
    return (1 - (float(l)/float(m))) * 100

In [18]:
print(levRatio(searchSTR, orgMatch))

print(levRatio(searchSTR, orgMatchLC))

15.151515151515149
27.27272727272727


In [19]:
print(fuzz.ratio(searchSTR, orgMatch))
print(fuzz.ratio(searchSTR, orgMatchLC))

19
35


### Fuzzy Wuzzy explanations
- fuzz.ratio() -> edit distance simply (levenshtein)
- fuzz.partial_ratio() -> checks for search str to be substr 
- fuzz.token_sort_ratio() -> alphabetically sort both string tokens and then match ratio
- fuzz.token_set_ratio() -> make set(intersect) from both strings and create combinations with both & match ratio

In [20]:
import re

In [21]:
# congOrgs = sorted(congOrgs)

In [22]:
print(congOrgs)

['JPMorgan Chase & Co' 'HSBC Bank'
 'Congressional Federal Credit Union Account' ...
 'Goldman Sachs/Canadian Dollars' 'Goldman Sachs/Australian Dollars'
 'BCIP Assoc III B']


In [23]:
results = [re.search("^[0-9]", l, flags=re.IGNORECASE) for l in congOrgs]

print(len(results))
# for i in range(len(congOrgs)): 
#     print(re.search('^a', congOrgs[i], flags=re.IGNORECASE))

70512


In [24]:
# print(results)

In [25]:
# results = list(filter(None.__ne__, results))
res = [i for i in range(len(results)) if not results[i] is None]
print([congOrgs[res[i]] for i in range(len(res))])

['401k Pension Plan American Funds', '401K /Pension Plan American Funds', '7344 Gwin St/Valley Springs CA', '7344 Gwin St/Valley Spring CA', '7400 Gwin St Valley/Springs, CA', '7344 Gwin St/Valley Springs, CA', '2200 acres land/Lindsay, TX', '292 acres land/Montague County, TX', '12 acres/Dallas, TX', '167 acres/Valley View, TX', '3M Co', '500 Bank Insured Deposit', '10 oz Gold Bar', '1290 Business DEV COM OPPTYS', '1303 TACT INC Closed-End', '1272 International High 30 DVD', '535 Marina Street/Rio Grande, PR', '615 Vistamar Street/Rio Grande, PR', '535 Marina St/Rio Grande, Puerto Rico', '615 Vistamar St/Rio Grande, Puerto Rico', '2025 W St Paul/Chicago, IL', '1003 W Fulton/Chicago, IL', '2146 W Churchill/Chicago, IL', '1 Lot/Columbia, SC', '1/3 Lot/Sumter, SC', '1/2 interest in two lots', '1/3 interest in unimproved land', '1 Lot', '1/3 Lot Gift/Sumter, SC', '529C', '113 Inherited Interest of Apartment in U', '6030 Woodrun Court/Montgomery, AL', '35 Bluff Drive LLC', '18th St/Tybee, 

#### Change the entries in congOrgs[res[i]] with '% '[1] to company names on a case by case basis and insert back where ever they're used in the actual dataframe

In [26]:
congDF[congDF['orgname'] == '15% interest in Legal Press, LLC']

,cid,personName,year,orgid,orgname,industry,asset_type,min,max,hidemax,type,lobbies,contributes,folderOfOrigin
177341,N00035607,Trott-Dave,2015,NaN,"15% interest in Legal Press, LLC",NaN,Cash/Money Market/Savings/Checking,1001,15000,0,Y,NaN,NaN,cong114


In [27]:
print(len(congDF['asset_type'].unique()))
congDF['asset_type'].unique()

9


array([nan, 'Cash/Money Market/Savings/Checking', 'Brokerage, IRA, 401k',
       'Government Bond', 'Unknown',
       '"Account" or Combination of other types (I.e.Stocks & Bonds)',
       'Insurance policies & Annuities', 'Mutual Fund',
       'Note/Note Receivable'], dtype=object)

In [28]:
# REGEX patterns (https://help.libreoffice.org/Common/List_of_Regular_Expressions)
pattern = re.compile('\+?%\s')  #testing  regex (https://www.regextester.com/94730)
matches = [x for x in congOrgs if pattern.findall(x)]

In [72]:
print(len(matches))

171


In [30]:
congDF[congDF['orgname'].isin(matches)].head()

,cid,personName,year,orgid,orgname,industry,asset_type,min,max,hidemax,type,lobbies,contributes,folderOfOrigin
3838,N00026710,Marchant-Kenny,2011,NaN,Ken-Ran Development 50% Interest Land,Unknown,NaN,50001,100000,0,Y,NaN,NaN,cong111
5479,N00006263,Simpson-Mike,2011,NaN,European Investment Bank Mtn 8.50000% 11,Unknown,NaN,28763,28763,0,Y,NaN,NaN,cong111
6611,N00012508,Carper-Tom,2012,NaN,Citigroup Funding 3% based on Gold,Unknown,Mutual Fund,15001,50000,0,Y,NaN,NaN,cong111
6738,N00012508,Carper-Tom,2011,NaN,Citigroup Funding 3% based on Gold,Unknown,Mutual Fund,15001,50000,0,Y,NaN,NaN,cong111
7498,N00012508,Carper-Tom,2010,NaN,Citigroup Funding 3% based on Gold,Unknown,Mutual Fund,50001,100000,0,Y,NaN,NaN,cong111


In [31]:
perList_sep = [s.split('% ') for s in matches]#map(lambda s: s.split('% ')[1], congDF['orgname'].values)

- It is important to note here that the '% 'tokenizer is quite confusing since the strings being tokenized are not     consistent. Some have comapny/property name after the tokenizer and some before.
- Therefore, the approach should be able to take all possible tokens (theoretically only two at max.) and check         matches

In [32]:
print(perList_sep[10]) 
# print(perList_sep[1])

['DB 100', 'PPN-ABS RTN BAR S&P 500']


In [33]:
perList_sep = []
for s in matches:
    temp = s.split('% ')
    perList_sep.append(temp[0]+temp[1])
#     if len(temp[0])>len(temp[1]) and len(temp[0].split(' '))>len(temp[1].split(' ')):
#         perList_sep.append(temp[0])
#     else:
#         perList_sep.append(temp[1])

In [34]:
matches.index('10% interest in Trott-ATA LLC')

121

In [35]:
print(perList_sep[16])

25interest in ZLDR partnership owning 95 acres +


##### Double string matching function for getting a very close match

In [36]:
print(process.extract(searchSTR, firmOrgsLC, limit=5))
print(fuzz.partial_ratio('CACI International', 'CACI International Inc'))

[('Alltel New York Inc', 86), ('Invesco Bond Fund', 86), ('VanEck Vectors Emerging Markets High Yield Bond ETF', 86), ('Gallery of History Inc', 86), ('New American Shoe Co', 86)]
100


In [37]:
def strMatch(searchVal, indexList, l1_thresh=85, l2_thresh=90, _size=5):
    matches = []
    baseList = process.extract(searchVal, indexList, limit=_size)
#     print('BASE_LIST: ',baseList)
    for _str, conf in baseList:
        if float(conf)>l1_thresh:
            match = fuzz.partial_ratio(searchVal, _str)
            if float(match)>l2_thresh:
                matches.append((_str, match))
    return baseList, matches

In [ ]:
searchSTR = 'JPMorgan 100US Treasury Securities Mon' # 'CACI International'
print(searchSTR,'\n\n')

BL, Matches = strMatch(searchSTR, firmOrgsLC)
print(BL)
print('\n')
print(Matches)
print('\n')
print('Highest Match:', Matches[0])

In [ ]:
print(fuzz.token_sort_ratio('JPMorgan 100', 'JPMorgan Chase & Co'))

In [ ]:
orgClean = []
tempDF = congDF[congDF['orgname'].isin(matches)]
for index, row in tempDF.iterrows():
    temp = row['orgname'].split('% ')
    orgClean.append(temp[0]+temp[1])
tempDF['orgClean'] = orgClean

In [ ]:
# matchIDX = []
# blIDX = []
# for index, row in congDF.iterrows():
#     BL, Matches = strMatch(row['orgname'], firmOrgsLC)
#     if Matches:
#         matchIDX.append(Matches[0])
#         blIDX.append(BL)
#     else:
#         matchIDX.append(np.nan)
#         blIDX.append(BL)

In [ ]:
# congDF['orgMatch'] = matchIDX
# congDF['orgBestOptions'] = blIDX

# tempDF.head(10)

In [ ]:
congDF.tail()

In [ ]:
congDF.to_csv('consolidatedMatched.csv')

### creating unique matches to reduce compute time

In [44]:
total_congOrgs = len(congOrgs)
i=0
match_dict = {}
baseList_dict = {}
for congFirm in congOrgs:
    BL, Matches = strMatch(congFirm, firmOrgsLC)
    
    baseList_dict[congFirm] = BL
    if Matches:
        match_dict[congFirm] = Matches[0]
        
    else:
        match_dict[congFirm] = np.nan
    
    #counter
    i+=1
    print((float(i)/total_congOrgs)*100)

0.001418198320853188
0.002836396641706376
0.004254594962559564
0.005672793283412752
0.007090991604265941
0.008509189925119127
0.009927388245972318
0.011345586566825504
0.012763784887678693
0.014181983208531881
0.01560018152938507
0.017018379850238255
0.018436578171091445
0.019854776491944635
0.02127297481279782
0.02269117313365101
0.0241093714545042
0.025527569775357386
0.026945768096210573
0.028363966417063763
0.02978216473791695
0.03120036305877014
0.03261856137962333
0.03403675970047651
0.035454958021329704
0.03687315634218289
0.03829135466303608
0.03970955298388927
0.04112775130474246
0.04254594962559564
0.04396414794644883
0.04538234626730202
0.046800544588155205
0.0482187429090084
0.04963694122986158
0.05105513955071477
0.05247333787156796
0.053891536192421145
0.05530973451327434
0.056727932834127526
0.05814613115498071
0.0595643294758339
0.06098252779668709
0.06240072611754028
0.06381892443839347
0.06523712275924666
0.06665532108009983
0.06807351940095302
0.06949171772180622
0.0

0.5970614930791922
0.5984796914000454
0.5998978897208985
0.6013160880417517
0.602734286362605
0.6041524846834581
0.6055706830043114
0.6069888813251645
0.6084070796460177
0.6098252779668709
0.611243476287724
0.6126616746085772
0.6140798729294304
0.6154980712502837
0.6169162695711369
0.61833446789199
0.6197526662128432
0.6211708645336964
0.6225890628545495
0.6240072611754027
0.6254254594962559
0.6268436578171092
0.6282618561379624
0.6296800544588155
0.6310982527796687
0.6325164511005219
0.6339346494213751
0.6353528477422283
0.6367710460630814
0.6381892443839347
0.6396074427047879
0.641025641025641
0.6424438393464942
0.6438620376673474
0.6452802359882006
0.6466984343090538
0.6481166326299069
0.6495348309507601
0.6509530292716134
0.6523712275924665
0.6537894259133198
0.6552076242341729
0.6566258225550261
0.6580440208758793
0.6594622191967324
0.6608804175175856
0.6622986158384389
0.6637168141592921
0.6651350124801453
0.6665532108009984
0.6679714091218516
0.6693896074427048
0.670807805763557

1.213977762650329
1.2153959609711822
1.2168141592920354
1.2182323576128886
1.2196505559337418
1.221068754254595
1.222486952575448
1.2239051508963013
1.2253233492171545
1.2267415475380077
1.228159745858861
1.229577944179714
1.2309961425005673
1.2324143408214205
1.2338325391422738
1.235250737463127
1.23666893578398
1.2380871341048332
1.2395053324256864
1.2409235307465396
1.2423417290673928
1.2437599273882458
1.245178125709099
1.2465963240299522
1.2480145223508055
1.249432720671659
1.2508509189925119
1.252269117313365
1.2536873156342183
1.2551055139550715
1.2565237122759247
1.257941910596778
1.259360108917631
1.2607783072384842
1.2621965055593374
1.2636147038801906
1.2650329022010438
1.266451100521897
1.2678692988427502
1.2692874971636035
1.2707056954844567
1.2721238938053099
1.2735420921261629
1.274960290447016
1.2763784887678693
1.2777966870887225
1.2792148854095757
1.2806330837304287
1.282051282051282
1.2834694803721352
1.2848876786929884
1.2863058770138416
1.2877240753346948
1.2891422

1.8337304288631722
1.8351486271840256
1.8365668255048786
1.8379850238257316
1.839403222146585
1.840821420467438
1.8422396187882915
1.8436578171091444
1.8450760154299977
1.8464942137508509
1.847912412071704
1.8493306103925573
1.8507488087134105
1.8521670070342635
1.853585205355117
1.85500340367597
1.8564216019968234
1.8578398003176764
1.8592579986385298
1.8606761969593828
1.8620943952802358
1.8635125936010892
1.8649307919219422
1.8663489902427957
1.8677671885636489
1.8691853868845019
1.8706035852053553
1.8720217835262083
1.8734399818470617
1.8748581801679147
1.8762763784887677
1.8776945768096212
1.8791127751304741
1.8805309734513276
1.8819491717721806
1.8833673700930338
1.884785568413887
1.8862037667347402
1.8876219650555934
1.8890401633764466
1.8904583616972996
1.891876560018153
1.893294758339006
1.8947129566598595
1.8961311549807125
1.8975493533015655
1.898967551622419
1.900385749943272
1.9018039482641254
1.9032221465849783
1.9046403449058316
1.9060585432266848
1.907476741547538
1.908

2.459155888359428
2.4605740866802814
2.4619922850011347
2.463410483321988
2.464828681642841
2.466246879963694
2.4676650782845475
2.4690832766054003
2.470501474926254
2.4719196732471067
2.47333787156796
2.474756069888813
2.4761742682096664
2.4775924665305196
2.479010664851373
2.480428863172226
2.481847061493079
2.4832652598139324
2.4846834581347856
2.486101656455639
2.4875198547764916
2.4889380530973453
2.490356251418198
2.4917744497390517
2.4931926480599045
2.4946108463807577
2.496029044701611
2.497447243022464
2.498865441343318
2.5002836396641706
2.5017018379850238
2.503120036305877
2.50453823462673
2.5059564329475834
2.5073746312684366
2.50879282958929
2.510211027910143
2.511629226230996
2.5130474245518495
2.5144656228727023
2.515883821193556
2.517302019514409
2.518720217835262
2.5201384161561156
2.5215566144769683
2.522974812797822
2.5243930111186748
2.525811209439528
2.527229407760381
2.5286476060812344
2.5300658044020876
2.531484002722941
2.532902201043794
2.5343203993646473
2.535

...

97.07851145904243
97.07992965736328
97.08134785568414
97.08276605400499
97.08418425232584
97.0856024506467
97.08702064896755
97.0884388472884
97.08985704560926
97.09127524393011
97.09269344225096
97.09411164057182
97.09552983889267
97.09694803721352
97.09836623553437
97.09978443385523
97.10120263217608
97.10262083049693
97.1040390288178
97.10545722713864
97.10687542545949
97.10829362378036
97.1097118221012
97.11113002042205
97.11254821874292
97.11396641706375
97.11538461538461
97.11680281370548
97.11822101202631
97.11963921034717
97.12105740866804
97.12247560698887
97.12389380530973
97.1253120036306
97.12673020195143
97.1281484002723
97.12956659859314
97.13098479691399
97.13240299523486
97.1338211935557
97.13523939187655
97.13665759019742
97.13807578851826
97.13949398683911
97.14091218515998
97.14233038348083
97.14374858180167
97.14516678012252
97.14658497844339
97.14800317676423
97.14942137508508
97.15083957340595
97.1522577717268
97.15367597004764
97.1550941683685
97.15651236668936
9

97.7308826866349
97.73230088495575
97.73371908327661
97.73513728159746
97.73655547991831
97.73797367823917
97.73939187656002
97.74081007488087
97.74222827320173
97.74364647152258
97.74506466984343
97.74648286816429
97.74790106648513
97.74931926480599
97.75073746312685
97.75215566144769
97.75357385976855
97.75499205808941
97.75641025641025
97.75782845473111
97.75924665305197
97.76066485137281
97.76208304969367
97.76350124801452
97.76491944633537
97.76633764465623
97.76775584297708
97.76917404129793
97.7705922396188
97.77201043793964
97.77342863626049
97.77484683458135
97.7762650329022
97.77768323122305
97.7791014295439
97.78051962786476
97.78193782618561
97.78335602450646
97.78477422282732
97.78619242114817
97.78761061946902
97.78902881778988
97.79044701611073
97.79186521443158
97.79328341275244
97.7947016110733
97.79611980939414
97.797538007715
97.79895620603585
97.8003744043567
97.80179260267757
97.80321080099841
97.80462899931926
97.80604719764013
97.80746539596097
97.80888359428182


98.3818357159065
98.38325391422737
98.38467211254822
98.38609031086906
98.38750850918993
98.38892670751078
98.39034490583163
98.39176310415249
98.39318130247334
98.39459950079419
98.39601769911505
98.3974358974359
98.39885409575675
98.40027229407761
98.40169049239846
98.40310869071931
98.40452688904017
98.40594508736102
98.40736328568187
98.40878148400273
98.41019968232358
98.41161788064443
98.41303607896529
98.41445427728614
98.41587247560699
98.41729067392784
98.4187088722487
98.42012707056955
98.4215452688904
98.42296346721126
98.42438166553211
98.42579986385296
98.42721806217382
98.42863626049467
98.43005445881552
98.43147265713638
98.43289085545722
98.43430905377808
98.43572725209894
98.43714545041978
98.43856364874064
98.4399818470615
98.44140004538234
98.4428182437032
98.44423644202406
98.4456546403449
98.44707283866576
98.44849103698661
98.44990923530746
98.45132743362832
98.45274563194917
98.45416383027002
98.45558202859088
98.45700022691173
98.45841842523258
98.45983662355344

99.03420694349899
99.03562514181982
99.03704334014068
99.03846153846155
99.03987973678238
99.04129793510324
99.04271613342411
99.04413433174494
99.0455525300658
99.04697072838667
99.0483889267075
99.04980712502837
99.05122532334921
99.05264352167006
99.05406171999093
99.05547991831178
99.05689811663262
99.05831631495349
99.05973451327434
99.06115271159518
99.06257090991605
99.0639891082369
99.06540730655774
99.0668255048786
99.06824370319946
99.0696619015203
99.07108009984115
99.07249829816202
99.07391649648287
99.07533469480371
99.07675289312458
99.07817109144543
99.07958928976628
99.08100748808714
99.08242568640799
99.08384388472884
99.0852620830497
99.08668028137055
99.0880984796914
99.08951667801226
99.09093487633311
99.09235307465396
99.09377127297482
99.09518947129567
99.09660766961652
99.09802586793737
99.09944406625823
99.10086226457908
99.10228046289993
99.10369866122079
99.10511685954164
99.10653505786249
99.10795325618335
99.1093714545042
99.11078965282505
99.11220785114591


99.68515997277059
99.68657817109144
99.6879963694123
99.68941456773315
99.690832766054
99.69225096437486
99.69366916269571
99.69508736101656
99.69650555933742
99.69792375765827
99.69934195597912
99.70076015429997
99.70217835262083
99.70359655094168
99.70501474926253
99.7064329475834
99.70785114590424
99.70926934422509
99.71068754254595
99.7121057408668
99.71352393918765
99.71494213750852
99.71636033582936
99.71777853415021
99.71919673247108
99.72061493079192
99.72203312911277
99.72345132743364
99.72486952575449
99.72628772407533
99.7277059223962
99.72912412071705
99.7305423190379
99.73196051735874
99.7333787156796
99.73479691400046
99.7362151123213
99.73763331064217
99.73905150896302
99.74046970728386
99.74188790560473
99.74330610392558
99.74472430224642
99.74614250056729
99.74756069888814
99.74897889720899
99.75039709552985
99.75181529385068
99.75323349217155
99.75465169049241
99.75606988881324
99.7574880871341
99.75890628545497
99.7603244837758
99.76174268209667
99.76316088041753
99.

In [45]:
print(len(match_dict))
print(len(baseList_dict))

70512
70512


In [46]:
set(match_dict.keys()) == set(baseList_dict.keys())

True

In [67]:
matchedDF = pd.DataFrame(match_dict.items(), columns=['congOrgs', 'match'])
baseListDF = pd.DataFrame(baseList_dict.items(), columns=['congOrgs', 'baseList'])

In [68]:
matchedDF.head()

,congOrgs,match
0,JPMorgan Chase & Co,"(JPMorgan Chase & Co, 100)"
1,HSBC Bank,NaN
2,Congressional Federal Credit Union Account,NaN
3,Chase Bank,"(Chevy Chase Bank FSB, 100)"
4,Citibank Accounts,NaN


In [69]:
baseListDF.head()

,congOrgs,baseList
0,JPMorgan Chase & Co,"[(JPMorgan Chase & Co, 100), (Acushnet Co, 86)..."
1,HSBC Bank,"[(American Bank Note Co, 86), (Bank of Hawaii ..."
2,Congressional Federal Credit Union Account,"[(Aetna Business Credit Inc, 86), (Allis-Chalm..."
3,Chase Bank,"[(Chevy Chase Bank FSB, 90), (American Bank No..."
4,Citibank Accounts,"[(CityBank, 79), (Citation Cos, 62), (Key Co, ..."


In [70]:
matchedDF.to_csv('matchedDF.csv')
baseListDF.to_csv('baseListDF.csv')

#### Weird company names

In [73]:
print('unique companies: ', len(congOrgs))

unique companies:  70512


In [74]:
pattern = re.compile('\+?%\s')  #testing  regex (https://www.regextester.com/94730)
matches = [x for x in congOrgs if pattern.findall(x)]

print('companies with "% " sub-string: ', len(matches))

companies with "% " sub-string:  171


In [75]:
pattern = re.compile('^[0-9]')  #testing  regex (https://www.regextester.com/94730)
matches = [x for x in congOrgs if pattern.findall(x)]

print('unique companies starting with number: ', len(matches))

unique companies starting with number:  2203
